# Project management

In [ ]:
from os import path, getenv
from mlrun import new_project

project_name = '-'.join(filter(None, ['properties-management']))
project_path = path.abspath('conf')
project = new_project(project_name, project_path, init_git=True)

print(f'Project path: {project_path}\nProject name: {project_name}')

In [ ]:
from mlrun import run_local, NewTask, mlconf, import_function, mount_v3io

# Target location for storing pipeline artifacts
artifact_path = path.abspath('jobs')
# MLRun DB path or API service URL
mlconf.dbpath = mlconf.dbpath or 'http://mlrun-api:8080'

print(f'Artifacts path: {artifact_path}\nMLRun DB path: {mlconf.dbpath}')

# Deploy code as an MLRun function

In [ ]:
from mlrun import code_to_function, mlconf,mount_v3io

In [ ]:
fn = code_to_function(project='Iguazio-Training',name='properities-handler', kind='job',filename='../modules/mlrun_functions.ipynb')

In [ ]:
fn.spec.image = "mlrun/mlrun"

In [ ]:
fn.apply(mount_v3io())

# Add function to project

In [ ]:
project.set_function(fn,name='properties-handler')

In [ ]:
project.functions

### Run interactive

In [ ]:
params = {"source_file" : "/User/igztraining/mlrun/data/demo.csv",
"target_file" : "/User/igztraining/mlrun/data/condos.csv" }

In [ ]:
save_properties = project.func('properties-handler').run(name='run_withcode',artifact_path=artifact_path,handler='save_properties',params=params,project='Iguazio-Training')

### Run step 2 with Step 1's output

In [ ]:
params['city']="SACRAMENTO"
params['target_dataset']="sacramento_condos"

In [ ]:
sp_run = project.func('properties-handler').run(name='run_withcode',artifact_path=artifact_path,handler='filter_properties',
                inputs={"source_data":save_properties.outputs["condo_properties"]},
                params=params,project='Iguazio-Training')

In [ ]:
project.save()